## 과제 3: SageMaker Processing 및 기본 제공 scikit-learn 컨테이너를 사용하여 데이터 처리 수행

이 노트북에서는 Amazon SageMaker Processing에서 제공 및 유지 관리하는 Docker 이미지를 사용하여 scikit-learn 스크립트를 실행하는 데 필요한 환경을 설정합니다. 

그런 다음 Amazon SageMaker Python SDK의 **SKLearnProcessor** 클래스를 사용하여 scikit-learn 처리 작업을 정의하고 실행합니다.

그리고 마지막으로 Amazon Simple Storage Service(Amazon S3)에 저장된 데이터 처리 결과를 검증합니다.

**참고**: 처리 스크립트는 중복 항목 제거, 대상 열을 레이블 2개가 포함된 열로 변환, 원-핫 인코딩, 훈련 및 검정 데이터 집합 생성을 위한 80-20 분할 등 몇 가지 기본적인 데이터 처리를 수행합니다. 

### 과제 3.1: 환경 설정

이 과제에서는 필요한 패키지와 종속성을 설치합니다. 

처리 작업의 출력을 저장할 Amazon S3 버킷을 설정하고 SageMaker Processing 작업을 실행할 실행 역할을 가져옵니다.

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np
from sagemaker.s3 import S3Downloader

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'labdatabucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

### 과제 3.2: SageMaker 처리 작업 실행

이 과제에서는 전처리된 데이터 집합을 가져와서 검토합니다.

In [ ]:
#import-data
prefix = 'data/input'

S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/adult_data.csv", local_path= 'data/')

shape=pd.read_csv("data/adult_data.csv", header=None)
shape.sample(5)

이 실습에서는 중복 항목 제거, 대상 열을 레이블 2개가 포함된 열로 변환, 범주별 특성의 원-핫 인코딩과 같은 데이터 변환을 수행합니다.

그런 다음, SKLearnProcessor 클래스를 생성하여 scikit-learn 처리 스크립트를 처리 작업으로 정의하고 실행합니다. 이 클래스에 관한 자세한 내용은 [SageMaker scikit-learn SKLearnProcessor](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#sagemaker.sklearn.processing.SKLearnProcessor)를 참조하세요.

SKLearnProcessor 클래스를 생성하려면 다음 파라미터를 구성합니다.
- **base_job_name**: 처리 작업 이름의 접두사
- **framework_version**: scikit-learn 버전
- **role**: SageMaker 실행 역할
- **instance_count**: 처리 작업을 실행할 인스턴스의 수
- **instance_type**: 처리 작업에 사용되는 Amazon Elastic Compute Cloud(Amazon EC2) 인스턴스의 유형

In [ ]:
#scikit-learn-processor
import os
from sagemaker.sklearn.processing import SKLearnProcessor

# create a SKLearnProcessor
sklearn_processor = SKLearnProcessor(
    base_job_name="sklearn-preprocessor",
    framework_version="0.20.0", 
    role=role, 
    instance_count=1,
    instance_type="ml.m5.xlarge"
)


다음으로는 **SKLearnProcessor.run()** 메서드를 사용하여 **sklearn_preprocessing.py** 스크립트를 처리 작업으로 실행합니다. 

처리 작업을 실행하기 위해 다음 파라미터를 구성합니다.
- **code**: 전처리 스크립트의 경로 
- **inputs 및 outputs**: 전처리 스크립트의 입력 및 출력 경로(Amazon S3 입력 및 출력 위치)
- **arguments**: 전처리 스크립트의 명령줄 인수(예: 훈련 및 검정 분할 비율)

처리 작업을 완료하려면 약 4-5분이 소요됩니다. 작업이 실행되는 동안 파일 브라우저에서 **sklearn_preprocessing.py** 파일을 열면 이 실습의 일부분으로 사전 구성된 전처리 스크립트의 소스를 검토할 수 있습니다.

In [ ]:
#processing-job
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "data/input"
output_preprocessed_data_prefix = "data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
sklearn_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "adult_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                         source="/opt/ml/processing/train",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                         source="/opt/ml/processing/test",
                         destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)

print("SKLearn Processing Job Completed.")

### 과제 3.3: 데이터 처리 결과 검증

이 과제에서는 실행한 처리 작업의 출력을 검증합니다. 이렇게 하려면 훈련 및 검정 출력 데이터 집합의 첫 5개 행을 검토합니다.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

### 마무리

축하합니다! SageMaker Processing에서 SageMaker Python SDK를 사용하여 scikit-learn 처리 작업을 생성했으며 해당 작업을 실행했습니다.

이 실습의 다음 과제에서는 SageMaker Processing을 사용하여 직접 빌드한 처리 컨테이너로 데이터 처리를 수행하는 방법을 중점적으로 살펴봅니다.

### 정리

이 노트북을 완료했습니다. 실습의 다음 부분으로 이동하려면 다음을 수행합니다.

- 이 노트북 파일을 닫습니다.
- 실습 세션으로 돌아가 **과제 4: 직접 빌드한 컨테이너로 데이터 처리 수행**을 계속 진행합니다.